### 导入数据，以第一类KPI 为例，整理成tsfresh format

In [1]:
import pandas

train_data=pandas.read_csv('../../data/train.csv')

In [2]:
train_data.columns

Index(['timestamp', 'value', 'label', 'KPI ID'], dtype='object')

### 原始训练数据set的数据标签y与数据x分割

In [3]:
train_data_x=train_data.drop(['label'],axis=1)
train_data_y=train_data['label']

### 取出一类KPI {采样频率为1分钟}

In [4]:
instance1_x=list(train_data_x.groupby(by='KPI ID'))[0][1]

In [5]:
instance1_y=train_data_y[:len(instance1_x)]

In [6]:
time_interval=60

### 将数据x 整理tsfresh格式

In [8]:
instance1_x['timestamp']=pandas.Series(range(0,len(instance1_x)))
instance1_x['KPI ID']=1

In [ ]:
start,end=-1,0
time_interval=60
for i in range(len(instance1_x)//time_interval):
    start=end+1
    end=start+time_interval-1
    if start>0:
        ef=extract_features(instance1_x[start:end], column_id="KPI ID", column_sort="timestamp")
        extracted_features=pandas.concat([extracted_features,ef],ignore_index=True)
    else:    
        extracted_features = extract_features(instance1_x[start:end], column_id="KPI ID", column_sort="timestamp")

sys.path.append('../utils')
import to_pickle as pk

pk.save_as_pickle('extracted_features.pickle',{'extracted_features':extracted_features})

### 对第一类KPI 的异常标志y 进行time_interval内的加权平均[非0窗口即为发生异常]

In [12]:
extracted_features=pk.load_pickle('extracted_features.pickle')['extracted_features']

In [8]:
instance1_y_=[]
for i in range(len(instance1_y)//time_interval):
    sum=0
    for j in range(time_interval):
        sum=sum+instance1_y[i*time_interval+j]
    instance1_y_.append(sum/time_interval)  
print(len(instance1_y_))    
print(len(instance1_y))  
sum=0
for k in range(len(instance1_y)-time_interval*len(instance1_y_)):
    sum=sum+instance1_y[time_interval*len(instance1_y_)+k]
instance1_y_.append(sum/len(instance1_y)-time_interval*len(instance1_y_))  
print(len(instance1_y_))  

2142
128562
2143


In [14]:
import numpy as np
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute

impute(extracted_features)
features_filtered = select_features(extracted_features,np.array(instance1_y_))

In [15]:
selected_feature=190

In [17]:
import numpy as np
trainX = np.array(features_filtered[:-2])
trainX=trainX[:,:selected_feature]
trainY = np.array(instance1_y_[:-2])
trainY=trainY
print(trainX.shape)
print(trainY.shape)

(2141, 190)
(2141,)


In [18]:
u=len(trainY[np.where(trainY>0)])
n=0
trainX_copy=[]
trainY_copy=[]
for i in range(trainX.shape[0]):
    if n<u  and trainY[i]==0:
        trainX_copy.append(trainX[i])
        trainY_copy.append(trainY[i])
        n=n+1;
    if trainY[i]!=0:
        trainX_copy.append(trainX[i])
        trainY_copy.append(trainY[i])
trainX_copy=np.array(trainX_copy)
trainY_copy = np.array(trainY_copy)        

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(trainX_copy, trainY_copy, test_size=0.2, random_state=123)

### 建立两层Dense+Dropout

In [20]:
import keras as K
from keras.models import Sequential
from keras.layers.core import Dense, Dropout


def buildModel(input_dim,drop=0.5,opt='adam',loss='mse',metrics=['accuracy']):
    model = Sequential()
    model.add(Dense(units=256,activation='relu',input_dim=input_dim))
    model.add(Dropout(drop))
    model.add(Dense(units=256,activation='relu'))
    model.add(Dropout(drop))
    model.add(Dense(units=1,activation='sigmoid'))
    model.compile(loss=loss,optimizer=opt,metrics=metrics)
    model.summary()
    return model

Using TensorFlow backend.


In [21]:
model=buildModel(selected_feature)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               48896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 114,945
Trainable params: 114,945
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.fit(x=X_train,y=y_train,batch_size=20,epochs=100,validation_data=(X_test,y_test))

Train on 361 samples, validate on 91 samples
Epoch 1/100
361/361 [==============================] - ETA: 10s - loss: 0.4554 - acc: 0.45 - ETA: 0s - loss: 0.5179 - acc: 0.3000 - 1s 2ms/step - loss: 0.4832 - acc: 0.3324 - val_loss: 0.3721 - val_acc: 0.4945
Epoch 2/100
361/361 [==============================] - ETA: 0s - loss: 0.4682 - acc: 0.400 - ETA: 0s - loss: 0.3738 - acc: 0.480 - ETA: 0s - loss: 0.3432 - acc: 0.486 - 0s 358us/step - loss: 0.3426 - acc: 0.4848 - val_loss: 0.3721 - val_acc: 0.4945
Epoch 3/100
361/361 [==============================] - ETA: 0s - loss: 0.3149 - acc: 0.500 - ETA: 0s - loss: 0.3810 - acc: 0.459 - 0s 269us/step - loss: 0.3659 - acc: 0.4654 - val_loss: 0.3721 - val_acc: 0.4945
Epoch 4/100
361/361 [==============================] - ETA: 0s - loss: 0.5372 - acc: 0.400 - ETA: 0s - loss: 0.3649 - acc: 0.453 - 0s 228us/step - loss: 0.3600 - acc: 0.4709 - val_loss: 0.3721 - val_acc: 0.4945
Epoch 5/100
361/361 [==============================] - ETA: 0s - loss: 0.2

361/361 [==============================] - ETA: 0s - loss: 0.2573 - acc: 0.600 - ETA: 0s - loss: 0.3254 - acc: 0.516 - 0s 222us/step - loss: 0.3401 - acc: 0.5014 - val_loss: 0.3721 - val_acc: 0.4945
Epoch 40/100
361/361 [==============================] - ETA: 0s - loss: 0.3928 - acc: 0.450 - ETA: 0s - loss: 0.3496 - acc: 0.487 - 0s 218us/step - loss: 0.3401 - acc: 0.5014 - val_loss: 0.3721 - val_acc: 0.4945
Epoch 41/100
361/361 [==============================] - ETA: 0s - loss: 0.1264 - acc: 0.750 - ETA: 0s - loss: 0.3312 - acc: 0.500 - 0s 252us/step - loss: 0.3401 - acc: 0.5014 - val_loss: 0.3721 - val_acc: 0.4945
Epoch 42/100
361/361 [==============================] - ETA: 0s - loss: 0.2212 - acc: 0.600 - ETA: 0s - loss: 0.3318 - acc: 0.504 - 0s 248us/step - loss: 0.3401 - acc: 0.5014 - val_loss: 0.3721 - val_acc: 0.4945
Epoch 43/100
361/361 [==============================] - ETA: 0s - loss: 0.2517 - acc: 0.550 - ETA: 0s - loss: 0.3528 - acc: 0.492 - 0s 244us/step - loss: 0.3401 - ac

361/361 [==============================] - ETA: 0s - loss: 0.2736 - acc: 0.550 - ETA: 0s - loss: 0.3279 - acc: 0.510 - 0s 228us/step - loss: 0.3401 - acc: 0.5014 - val_loss: 0.3721 - val_acc: 0.4945
Epoch 78/100
361/361 [==============================] - ETA: 0s - loss: 0.2405 - acc: 0.650 - ETA: 0s - loss: 0.3455 - acc: 0.492 - 0s 246us/step - loss: 0.3401 - acc: 0.5014 - val_loss: 0.3721 - val_acc: 0.4945
Epoch 79/100
361/361 [==============================] - ETA: 0s - loss: 0.4213 - acc: 0.400 - ETA: 0s - loss: 0.3459 - acc: 0.513 - 0s 295us/step - loss: 0.3401 - acc: 0.5014 - val_loss: 0.3721 - val_acc: 0.4945
Epoch 80/100
361/361 [==============================] - ETA: 0s - loss: 0.4443 - acc: 0.300 - ETA: 0s - loss: 0.3218 - acc: 0.504 - 0s 305us/step - loss: 0.3401 - acc: 0.5014 - val_loss: 0.3721 - val_acc: 0.4945
Epoch 81/100
361/361 [==============================] - ETA: 0s - loss: 0.3759 - acc: 0.400 - ETA: 0s - loss: 0.3357 - acc: 0.481 - 0s 295us/step - loss: 0.3401 - ac

### 导自写工具

In [11]:
import sys
sys.path.append('../utils')
import to_pickle as pk

In [11]:
import sys
sys.path.append(u'E:\\a_school\智能运维\code\朱嘉业-1352953')

In [12]:
import util

Using TensorFlow backend.
